In [1]:
from IPython.core.display import HTML, Javascript
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [2]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

# Spielen gegen die KI

## Vom Nutzer zu tätigende Einstellungen
Zur Erstellung eines individuellen Spielfeldes werden Dictionaries verwendet. Diese verwenden jeweils als `key` die Figur, 
die auf dem Spielfeld platziert werden soll. Als `value` nutzen die Einträge jeweils eine Liste, 
die vom Nutzer mit den String-Bezeichnern der Felder gefüllt werden sollen, auf denen die jeweilige Figur steht.
Die Bezeichner folgen hierbei dem Format, welches in Kapitel: TODO erklärt wurde.

In [3]:
WHITE_POSITIONS = {'king':['e1'],
                   'queen':[],
                   'rooks':['a1'],
                   'bishops':[],
                   'knights':[],
                   'pawns':[]}

BLACK_POSITIONS = {'king':['e7'],
                   'queen':[],
                   'rooks':[],
                   'bishops':[],
                   'knights':[],
                   'pawns':[]}

Weiter muss der Nutzer angeben, welche vorher berechnete Spielsituation er laden möchte.
Hierfür muss der Dateiname in einer globalen Variable (ohne Dateiendung `.chessAI`) angegeben werden.

In [4]:
FILE = "S_n_seq_rook"

## Logik für die Interaktion mit der Spielsituation
Im folgenden Abschnitt sollen einige Hilfsfunktionen erklärt werden, welche für die Interaktion zwischen dem Nutzer über ein Jupyter Notebook und 
der Spielsituation benötigt werden.  

Die Funktion `get_occupied_cells()` übersetzt die zuvor erstellten und vom Nutzer veränderten `Position-Dictionaries` in eine Liste, die alle ``values`` der
Dictionaries, also besetzte Felder enthält. Diese Liste wird benötigt, um zu überprüfen, ob die Eingaben des Nutzers korrekte Zellen sind. 

In [5]:
def get_occupied_cells():
    cells = []
    for values in WHITE_POSITIONS.values():
        for value in values:
            cells.append(value)
    for values in BLACK_POSITIONS.values():
        for value in values:
            cells.append(value)
    return cells

Mit der Information von der vorherigen Funktion übernimmt die Funktion `check_for_correct_cells()` die Überprüfung. 
Hierzu wird jeder Wert in den eingegebenen Feldern in den Dictionaries `WHITE_POSITIONS` und `BLACK_POSITIONS` auf einhaltung
eines regulären Ausdrucks, welcher auf eine Kombination aus Buchstabe und Zahl überprüft. 
Das Ergebnis wird in Form eines booleschen Werts zurückgegeben. 

In [6]:
#TODO: move
import re
def check_for_correct_cells():
    cells = get_occupied_cells()
    for cell in cells:
        x = re.search("[a-h][1-8]", cell)
        if x is None:
            print(cell)
            print("Value incorrect!")
            return False
        else:
            pass
    return True

TODO: Könnten wir die folgende Funktion nicht einfach dadurch ersetzen, die `chess.*` Schreibweise im Dictionary zu verwenden?

Nachdem die Eingaben des Nutzers validiert wurden, muss die Situation auf einem Board-Objekt der `chess` Library umgesetzt werden.
Zunächst müssen die `keys` des vom Nutzer veränderten Dictionaries zu Werten der Bibliothek gewandelt werden. 
Die Funktion `get_piece_type_by_name(name)` übersetzt einen String mit dem Namen einer Figur (z.B. "king") in einen `PIECE_TYPE` der `chess` Library. 

In [7]:
def get_piece_type_by_name(name):
    if name == 'king':
        piece_type = chess.KING
    elif name == 'queen':
        piece_type = chess.QUEEN
    elif name == 'rooks':
        piece_type = chess.ROOK
    elif name == 'bishops':
        piece_type = chess.BISHOP
    elif name == 'knights':
        piece_type = chess.KNIGHT
    else:
        piece_type = chess.PAWN
    return piece_type

Bevor die Figuren dem Objekt hinzugefügt werden, werden sie in einem Dictionary, welches als `Piece-Map` verwendet wird gesammelt.

Die Funktion `collect_cells(color, pieces)` erstellt aus einem Boolschen-Wert für die Spielerfarbe und einem Dictionary mit
Figuren und Spielfeldern eine Liste mit `Piece` 
Objekten, welche als Index den Index des Spielfeldes verwendet. Dise Liste entspricht dem Format mit welchem Formationen von der
Bibliothek geladen werden können.  

In [8]:
def collect_cells(color, pieces):
    occupied_cells = {}
    for key, values in pieces:
            piece_type = get_piece_type_by_name(key)
            piece = chess.Piece(piece_type, color)
            for value in values:
                square = chess.parse_square(value)
                occupied_cells[square] = piece
            
    return occupied_cells

Die Funktion `create_board()` füllt das Schachbrett letztlich mit den Figuren, die in den Dictionaries angegeben wurden. Als
Rückgabewert gibt die Funktion das gefüllte Schachbrett als Objekt der `chess` Library zurück.
Hierzu werden die vorherigen Funktionen verwendet, um die `Piece-Map` zu erstellen, diese 
auf das Board-Objekt angewandt und auf Validität überprüft.
Ist das Board nicht valide, wird ein Fehler ausgegeben.

Ein Problem, welches in Spielsituationen mit dem Turm auftritt, ist die Berechtigung zu einer Rochade.
Damit das Board als valide anerkannt wird, werden die entsprechenden Flags mit der Funktion
`clean_castling_rights` korrekt gesetzt.

In [9]:
def create_board():
    local_board = chess.Board()
    occupied_cells = {}
    if check_for_correct_cells():
        occupied_cells |= collect_cells(chess.WHITE, WHITE_POSITIONS.items())
        occupied_cells |= collect_cells(chess.BLACK, BLACK_POSITIONS.items())

        local_board.set_piece_map(occupied_cells)
        local_board.castling_rights = local_board.clean_castling_rights()
        
        # Herausfinden, weshalb momentan angegebenes Board invalid ist
        if not local_board.is_valid():
            display(local_board)
            print("Specified lineup is invalid")
            print("Default board created instead")
            local_board = chess.Board()
    return local_board

## Import der Daten

Für die Bestimmung der Züge der KI werden die $S_n$ Mengen verwendet, die im Notebook ``calculation.ipynb`` berechnet werden.   
Eine Erklärung worum es sich hierbei handelt, findet sich in diesem Notebook.
Die Ergebnisse der Berechnung werden mittels `pickle` serialisiert und in einer ZIP-Datei komprimiert abgespeichert. Zur verwendung wird das Archiv entpackt und 
die Liste deserialisiert. 
Weitere Informationen zum Inhalt der Datei befinden sich ebenfalls im Notebook ``calculation.ipynb``.   

Die Funktion `load_s_n_sequence(filename)` erhält den Dateinamen (``filename``) als Parameter und gibt die Endspieldaten zurück.

In [10]:
def load_s_n_sequence(filename):
    s_n_sequence_tuples = []
    with ZipFile("S_n_Results/" + filename + ".chessAI") as zipped:
        with zipped.open(filename + ".pickle") as calculation:
            tmp = pickle.loads(calculation.read())
            for item in tmp:
                s_n_sequence_tuples.append(item)
    return s_n_sequence_tuples

## Den besten Zug für die KI ermitteln
Um mit den berechneten Endspieldaten gegen einen Spieler zu gewinnen, muss jeder Zug, den weiß macht, optimal sein.
Ein passender Zug für die KI besteht darin die Situation von $S_n$ in einen Zustand zu überführen, in dem sie sich in $S_{n-1}$ befindet.  

Die Funktion `find_next_move(curr_board, s_index, s_n_sequence)` bestimmt für ein übergebenes Board-Objekt einen solchen Spielzug. 
Aus Effizienzgründen wird zusätzlich zum gefundenen Spielzug (`move`) der neue Wert für $n$ zurückgegeben. 

Für diese Berechnung benötigt ``find_next_move`` nachkommende Argumente:
* ``curr_board``: Das Board, für welches der nächste Spielzug berechnet werden soll.
* ``s_index``: Das $n$ eines $S_n$, in welchem sich ``curr_board`` befindet. Wird aus Effizienzgründen übergeben.
* ``s_n_sequence``: Die Liste mit allen $S_n$

Kann kein Spielzug gefunden werden, gibt die Funktion den Wert -1 zurück.

Die für diesen Ablauf benötigte Funktion `find_situation_in_sequence` wird im Notebook `functions.ipynb` definiert und erklärt.

In [11]:
def find_next_move(curr_board, s_index, s_n_sequence):
    s_index_new = s_index - 1
    for move in curr_board.legal_moves:
        curr_board.push(move)
        curr_tupel = (curr_board.turn, curr_board.__str__())
        _tmp = find_situation_in_sequence(curr_tupel, [s_n_sequence[s_index_new]])
        if _tmp != -1:
            curr_board.pop()
            return s_index_new, move
        curr_board.pop()

    return -1, None

## Globale Variablen für die Anzeige

Nachdem alle Funktionen für die Bestimmung eines Zugs definiert worden sind, gilt es die bereitgestellte UI der ``python-chess`` Bibliothek zu erweitern. 
In Form von globalen Variablen werden UI-Elemente definiert, die für die Eingabe eines neuen Zugs benötigt werden:

- `input_field` = Ein Eingabefeld, in dem der nächste zug von weiß eingetragen werden soll.
- `execute_button` = Ein Button, der nach der Auswahl des Zuges diesen auch auf dem Schachbrett ausführt.

In [12]:
input_field = widgets.Text()

execute_button = widgets.Button(
    description='Execute Move',
    disabled=False,
    button_style='',
    tooltip='Executes the move selected with the dropdowns "Piece:" and "Move:"',
    icon='check'
)

## Globale Variablen für den aktuellen Spielzustand
Damit der aktuelle Stand des Spiels auch ohne eine lineare Kette von Funktionsaufrufen verfügbar ist, 
werden diese Informationen in globalen Variablen gespeichert.

Weiterhin werden weitere globale Variablen definiert, die einerseits den Dateinamen für die Spielzüge (`filename`), 
andererseits das Spielbrett zum Spielen gegen die KI innehalten (`board`).

In [13]:
def get_color(turn):
    if turn:
        return "White"
    else:
        return "Black"

Innerhalb der python-chess library werden die Farben des Schachbretts `chess.WHITE` und `chess.BLACK` als boolesche Variablen definiert. 
Deswegen wurde zur späteren Dokumentation, aber auch zum Debuggen eine Funktion geschrieben, 
die die Farbe, die gerade am Zug ist, als String zurückgibt.

In [14]:
S_N_TUPLES = load_s_n_sequence(FILE)
S_INDEX = 0
BOARD_INDEX = 0

Weiterhin werden weitere globale Variablen definiert, die einerseits den Dateinamen für die Spielzüge (`filename`), 
andererseits das Spielbrett zum Spielen gegen die KI innehalten (`board`).